In [4]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

def process_race_data(input_file_path, output_file_path):
        # Read the Excel file
    df = pd.read_excel(input_file_path)
    # Sort the data by RunnerID and DateOfTheRace
    df.sort_values(by=['RunnerID', 'DateOfTheRace'], ascending=[True, True], inplace=True)
    
    # Create a new DataFrame to store the results
    output_data = pd.DataFrame(columns=[ 'Marathon','RaceOne','RaceOneKM','RaceOneResultInSecondsNormalized',
                                        'TimeBetweenMarathonAndRaceOne','RaceTwo','RaceTwoKM',
                                        'RaceTwoResultInSecondsNormalized','TimeBetweenRaceOneAndRaceTwo',
                                        'MarathonResultInSeconds','MarathonResultInSecondsNormalized','ImprovementRate',
                                        'ImprovementClass',
                                        'RunnerID','MarathonDate', 'MarathonResult', 'RaceOneResult', 'RaceOneResultInSeconds',
                                        'RaceTwoResult', 'RaceTwoResultInSeconds'])
    
    # Iterate over unique RunnerIDs\
    max_time_per_race = df.groupby(['NameOfTheRace', 'NumberOfMeters'])['Result'].max()
    min_time_per_race = df.groupby(['NameOfTheRace', 'NumberOfMeters'])['Result'].min()

    for runner_id in df['RunnerID'].unique():
        runner_data = df[df['RunnerID'] == runner_id]

        # Find the marathon (race with distance > 40000 meters)
        marathon_data = runner_data[runner_data['NumberOfMeters'] > 40000].tail(1)
        if marathon_data.empty:
            continue
    
        marathon_name =  marathon_data['NameOfTheRace'].values[0]
        marathon_date =  marathon_data['DateOfTheRace'].values[0] 
        marathon_result = marathon_data['Result'].values[0]
        marathon_result_seconds = time_to_seconds(marathon_result)
        marathon_date = marathon_data['DateOfTheRace'].values[0]

        # Find the races before the marathon
        races_before_marathon = runner_data[runner_data['DateOfTheRace'] < marathon_date ]

        if len(races_before_marathon) >= 2:
            race_one = races_before_marathon.tail(1)
            race_two = races_before_marathon.tail(2).head(1)

            race_one_name = race_one['NameOfTheRace'].values[0]
            race_one_result = race_one['Result'].values[0]
            race_one_result_seconds = time_to_seconds(race_one_result)
            race_one_distance = race_one['NumberOfMeters'].values[0]
            race_one_date = race_one['DateOfTheRace'].values[0]

            race_two_name = race_two['NameOfTheRace'].values[0]
            race_two_result = race_two['Result'].values[0]
            race_two_result_seconds = time_to_seconds(race_two_result)
            race_two_distance = race_two['NumberOfMeters'].values[0]
            race_two_date = race_two['DateOfTheRace'].values[0]
            # Normalization calculation
            min_race_one = time_to_seconds(min_time_per_race[(race_one_name,race_one_distance)])
            min_race_two = time_to_seconds(min_time_per_race[(race_two_name,race_two_distance)])
            min_marathon =  time_to_seconds(min_time_per_race[(marathon_name,42195)])
            
            range_race_one =  time_to_seconds(max_time_per_race[(race_one_name,race_one_distance)]) - min_race_one
            range_race_two = time_to_seconds(max_time_per_race[(race_two_name,race_two_distance)]) - min_race_two
            range_marathon = time_to_seconds(max_time_per_race[(marathon_name,42195)]) - min_marathon
            if range_race_one == 0 or range_marathon == 0 or range_race_two == 0:
                # to not devide by zero
                continue
            race_one_normalized = (race_one_result_seconds - min_race_one) / range_race_one
            race_two_normalized = (race_two_result_seconds - min_race_two) / range_race_two 
            marathon_normalized = (marathon_result_seconds - min_marathon) / range_marathon

            time_between_marathon_and_race_one = (race_one_date - marathon_date).astype('timedelta64[D]')/np.timedelta64(1, 'D')
            time_between_race_one_and_race_two = (race_two_date - race_one_date).astype('timedelta64[D]')/np.timedelta64(1, 'D')
        else:
            # If there are no two races before the marathon, skip this runner
            continue

        if time_between_marathon_and_race_one + time_between_race_one_and_race_two <= -120 :
            # if the time range between the marathon and race two is more than 4 months - it's not preperation for marathon
            continue
        output_data = pd.concat([output_data, pd.DataFrame([{
            'RaceOneResultInSecondsNormalized': race_one_normalized,
            'RaceTwoResultInSecondsNormalized': race_two_normalized,
            'MarathonResultInSecondsNormalized': marathon_normalized, 
            'RunnerID': runner_id,
            'Marathon': marathon_name,
            'MarathonDate': marathon_date,
            'MarathonResult': marathon_result,
            'MarathonResultInSeconds': marathon_result_seconds,
            'RaceOne': race_one_name,
            'RaceOneResult': race_one_result,
            'RaceOneKM': race_one_distance,
            'RaceOneResultInSeconds': race_one_result_seconds,
            'TimeBetweenMarathonAndRaceOne': time_between_marathon_and_race_one,
            'RaceTwo': race_two_name,
            'RaceTwoResult': race_two_result,
            'RaceTwoKM': race_two_distance,
            'RaceTwoResultInSeconds': race_two_result_seconds,
            'TimeBetweenRaceOneAndRaceTwo': time_between_race_one_and_race_two
        }])], ignore_index=True)
        
         # Calculate the maximum values for normalization
        #max_marathon_result = output_data['MarathonResultInSeconds'].max()
        #max_race_one_result = output_data['RaceOneResultInSeconds'].max()
        #max_race_two_result = output_data['RaceTwoResultInSeconds'].max()

        # Calculate normalized values for MarathonResultInSeconds, RaceOneResultInSeconds, and RaceTwoResultInSeconds
        #output_data['MarathonResultInSecondsNormalized'] = output_data['MarathonResultInSeconds'] / max_marathon_result
        #output_data['RaceOneResultInSecondsNormalized'] = output_data['RaceOneResultInSeconds'] / max_race_one_result
        #output_data['RaceTwoResultInSecondsNormalized'] = output_data['RaceTwoResultInSeconds'] / max_race_two_result
    
    output_data['ImprovementRate'] = output_data['MarathonResultInSecondsNormalized'] - ( output_data['RaceOneResultInSecondsNormalized'] + output_data['RaceTwoResultInSecondsNormalized']) /2
    #output_date['ImprovementClass'] = 'Success' if output_data['ImprovementRate'] < 0 else 'Not Success' 
    
    output_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')


def time_to_seconds(time_str):
    # Convert a time string in format 'hours:minutes:seconds' to total seconds
    time_str = time_str.strftime("%H:%M:%S")
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

# Example usage:
input_file_path = 'InputToProcess.xlsx'
output_file_path = 'OutputFromProcess.csv'


process_race_data(input_file_path, output_file_path)


In [ ]:
print(max_time_per_race[('Jerusalem Marathon', 21097)])